In [ ]:
from dash import dcc, html, Input, Output, State, Dash, callback_context
import plotly.express as px
import pandas as pd
import numpy as np
import calendar
import datetime

# Incorporate data
df = pd.read_csv('data.csv')

# Create Date Columns
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Year'] = df['Timestamp'].dt.year
df['Month'] = df['Timestamp'].dt.month.apply(lambda x: calendar.month_name[int(x)])
df['Day'] = df['Timestamp'].dt.date
df['Hour'] = df['Timestamp'].dt.hour
df['Minute'] = df['Timestamp'].dt.strftime('%H:%M')



app = Dash(__name__)

app.layout = html.Div([
    html.H2("Ferry Redemptions Drilldown"),
    html.Button("Back", id="back-button", n_clicks=0),
    dcc.Graph(id="bar-graph")
])

# App state to track drilldown level
drill_levels = ['Year', 'Month', 'Day', 'Hour', 'Minute']
drill_titles = [
    'Total Redemptions by Year',
    'Monthly Redemptions in {}',
    'Daily Redemptions in {} {}',
    'Hourly Redemptions on {}',
    'Minute-Level Redemptions on {}'
]

drill_state = []

@app.callback(
    Output("bar-graph", "figure"),
    Input("bar-graph", "clickData"),
    Input("back-button", "n_clicks"),
    State("bar-graph", "figure")
)
def update_graph(clickData, n_clicks, current_fig):
    global drill_state
    ctx = callback_context

    if not ctx.triggered:
        trigger = None
    else:
        trigger = ctx.triggered[0]['prop_id'].split('.')[0]

    if trigger == "back-button" and drill_state:
        drill_state.pop()

    elif trigger == "bar-graph" and clickData is not None and len(drill_state) < 4:
        clicked_value = str(clickData['points'][0]['x'])
        drill_state.append(clicked_value)

    # Determine drilldown level
    level = len(drill_state)

    if level == 0:
        df_grouped = df.groupby("Year")['Redemption Count'].sum().reset_index()
        fig = px.bar(df_grouped, x="Year", y="Redemption Count", title=drill_titles[0])
        fig.update_xaxes(type='category')
        return fig

    elif level == 1:
        df_filtered = df[df['Year'] == int(drill_state[0])]
        df_grouped = df_filtered.groupby("Month")['Redemption Count'].sum().reset_index()
        fig = px.bar(df_grouped.sort_values(by="Month"), x="Month", y="Redemption Count", title=drill_titles[1].format(drill_state[0]))
        fig.update_xaxes(type='category')
        return fig

    elif level == 2:
        df_filtered = df[(df['Year'] == int(drill_state[0])) & (df['Month'] == drill_state[1])]
        df_grouped = df_filtered.groupby("Day")['Redemption Count'].sum().reset_index()
        fig = px.bar(df_grouped.sort_values(by="Day"), x="Day", y="Redemption Count", title=drill_titles[2].format(drill_state[1], drill_state[0]))
        fig.update_xaxes(type='category')
        return fig

    elif level == 3:
        df_filtered = df[df['Day'] == datetime.datetime.strptime(drill_state[2], "%Y-%m-%d").date()]
        df_grouped = df_filtered.groupby("Hour")['Redemption Count'].sum().reset_index()
        fig = px.bar(df_grouped, x="Hour", y="Redemption Count", title=drill_titles[3].format(drill_state[2]))
        fig.update_xaxes(type='category')
        return fig

    elif level == 4:
        df_filtered = df[
            (df['Day'] == datetime.datetime.strptime(drill_state[2], "%Y-%m-%d").date()) &
            (df['Hour'] == int(drill_state[3]))
    ]
        df_grouped = df_filtered.groupby("Timestamp")['Redemption Count'].sum().reset_index()
        df_grouped["label"] = df_grouped["Timestamp"].dt.strftime('%H:%M')
        fig = px.bar(df_grouped, x="label", y="Redemption Count", title=drill_titles[4].format(drill_state[2]))
        fig.update_xaxes(type='category')
        return fig

    return dash.no_update

if __name__ == '__main__':
    app.run(debug=True)


: 